In [107]:
import importlib
import labs
import pred_llv3
import llv3
import inv 

importlib.reload(labs)
importlib.reload(pred_llv3)
importlib.reload(llv3)
importlib.reload(inv)

from labs import *
from inv import *
from pred_llv3 import *
from llv3 import *

# Predictor

In [2]:
image_paths = grep_files('/data/ephemeral/home/dataset/docsy-deskew/', exts=['jpg'])
len(image_paths)

0it [00:00, ?it/s]

3140

In [3]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

In [80]:
ckpt_path = './ckpt/llv3-easy-rotator-subclasses-480hw-last_epoch.ckpt'
classes_path = '/data/ephemeral/home/dataset/dtc/doc_classes.json'
predictor = Predictor(Lym, processor, ckpt_path, classes_path)

Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# TEST

In [56]:
outs, preds = predictor.feed(image_paths[0])

/data/ephemeral/home/.pyenv/versions/py12/lib/python3.12/site-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


# TEST ALL

In [68]:
items = predictor.test(image_paths[:])

  0%|          | 0/3140 [00:00<?, ?it/s]

In [69]:
write_list('./llv3-dice-loss-sub-classes.txt', items)

# Compare

In [70]:
wrongs = read_list_from_txt('inconsist_list.txt')
base_dir = '/data/ephemeral/home/dataset/docsy-deskew/'
paths = [os.path.join(base_dir, i) for i in wrongs]

In [94]:
answers = [(item[0][:-4] + '.jpg', np.argmax(np.array(item[1])), np.array(item[1]).max()) for item in items]
df = pd.DataFrame(answers, columns=['ID', 'target', 'prob'])
hists = [ df ]

In [108]:
class_names = predictor.class_names
def pick(filename):
    x = [item for item in items if item[0] == filename][0]
    probs = x[1]
    row = [float("{:.4f}".format(p)) for p in probs] 
    columns = [f'{name}({i})' for i, name in enumerate(class_names)]
    df = pd.DataFrame([row], columns=columns)
    return df

In [109]:
pick('279c0046ee09258d.jpg')

,계좌번호(0),임신/출산(1),계기판(2),입퇴원 확인서(3),진단서(4),면허증(5),의료비 영수증(6),진료/통원 확인서(7),주민등록증(8),여권(9),납입 확인서(10),약국 영수증(11),처방전(12),이력서(13),소견서(14),차량등록증(15),자동차(16)
0,0.0001,0.0004,0.0001,0.1276,0.0042,0.0001,0.0001,0.6749,0.0001,0.0001,0.0001,0.0001,0.0001,0.0006,0.1915,0.0001,0.0001


In [95]:
inv = ImageNavigator(paths, hists, predictor.id2kor)